In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import re

#### Import the data and get a high-level picture

In [11]:
df = pd.read_csv('sales.csv')
df.head()

,order_id,name,ordered_at,price,quantity,line_total
0,10000,"""ICE CREAM"" Peanut Fudge",2018-01-01 11:30:00,$3.50,3,$10.50
1,10000,"""ICE CREAM"" Peanut Fudge",2018-01-01 11:30:00,$3.50,1,$3.50
2,10001,"""SORBET"" Raspberry",2018-01-01 12:14:54,$2.50,2,$5.00
3,10001,NaN,2018-01-01 12:14:54,$1.50,1,$1.50
4,10001,"""CONE"" Dipped Waffle Cone",2018-01-01 12:14:54,$3.50,1,$3.50


In [3]:
df.shape

(29922, 6)

In [4]:
df.dtypes

order_id       int64
name          object
ordered_at    object
price         object
quantity       int64
line_total    object
dtype: object

#### TODO: Fix column datatypes

Change ordered_at to datetime

Change price and line_total to float

In [12]:
df["ordered_at"] = pd.to_datetime(df["ordered_at"])

In [15]:
df["price"] = df["price"].replace({r'\$':''}, regex = True)
df["line_total"] = df["line_total"].replace({r'\$':''}, regex = True)

df.head()
df["price"] = df["price"].astype("float")
df["line_total"] = df["line_total"].astype("float")

In [16]:
df.dtypes

order_id               int64
name                  object
ordered_at    datetime64[ns]
price                float64
quantity               int64
line_total           float64
dtype: object

#### TODO: drop if duplicated or null

In [6]:
df[df.duplicated()].shape[0]

538

In [17]:
df=df.drop_duplicates()
df[df.duplicated()].shape[0]

0

In [18]:
df.isnull().sum()

order_id         0
name          1481
ordered_at       0
price            0
quantity         0
line_total       0
dtype: int64

In [19]:
df[df['name'].isnull()].head()

,order_id,name,ordered_at,price,quantity,line_total
3,10001,NaN,2018-01-01 12:14:54,1.5,1,1.5
6,10002,NaN,2018-01-01 12:23:09,3.0,3,9.0
27,10007,NaN,2018-01-01 15:03:17,2.5,1,2.5
77,10026,NaN,2018-01-02 03:25:40,0.5,2,1.0
88,10031,NaN,2018-01-02 05:45:48,3.5,3,10.5


In [20]:
df=df.dropna()
print(df)


       order_id                       name          ordered_at  price  \
0         10000   "ICE CREAM" Peanut Fudge 2018-01-01 11:30:00    3.5   
1         10000   "ICE CREAM" Peanut Fudge 2018-01-01 11:30:00    3.5   
2         10001         "SORBET" Raspberry 2018-01-01 12:14:54    2.5   
4         10001  "CONE" Dipped Waffle Cone 2018-01-01 12:14:54    3.5   
5         10002            "SORBET" Lychee 2018-01-01 12:23:09    3.0   
...         ...                        ...                 ...    ...   
29817     19997         "CONE" Waffle Cone 2018-07-28 17:40:40    4.0   
29818     19997      "SORBET" Blood Orange 2018-07-28 17:40:40    2.5   
29819     19998            "SORBET" Lychee 2018-07-28 18:21:44    3.0   
29820     19998     "ICE CREAM" Rocky Road 2018-07-28 18:21:44    3.5   
29821     19999      "SORBET" Blood Orange 2018-07-28 18:51:57    2.5   

       quantity  line_total  
0             3        10.5  
1             1         3.5  
2             2         5.0  
4  

#### Sanity check for value ranges and to check assumptions

In [9]:
df[(df['price'] * df['quantity']) != df['line_total']].shape[0]

19924

In [10]:
df[df['line_total'] < 0].shape[0]

TypeError: '<' not supported between instances of 'str' and 'int'

#### TODO: 
Set line_total = price * quantity if different
Remove if line total < 0

In [21]:
if df[(df['price'] * df['quantity']) != df['line_total']].shape[0]:df['line_total'] = df['price'] * df['quantity']
df[(df['price'] * df['quantity']) != df['line_total']].shape[0]

0

In [25]:
if (df[df['line_total'] < 0].shape[0]):
    df_temp = df[ df['line_total']< 0 ].index
    df.drop(df_temp , inplace=True)
    
df[df['line_total'] < 0].shape[0]

0

In [ ]:
df.describe()

#### TODO: Get value between "" in name and put it in category column

In [26]:
df = df.assign(category = lambda x: (df.name.str.extract(r'\"(.+?)\"', expand=False)))

df

,order_id,name,ordered_at,price,quantity,line_total,category
0,10000,"""ICE CREAM"" Peanut Fudge",2018-01-01 11:30:00,3.5,3,10.5,ICE CREAM
1,10000,"""ICE CREAM"" Peanut Fudge",2018-01-01 11:30:00,3.5,1,3.5,ICE CREAM
2,10001,"""SORBET"" Raspberry",2018-01-01 12:14:54,2.5,2,5.0,SORBET
4,10001,"""CONE"" Dipped Waffle Cone",2018-01-01 12:14:54,3.5,1,3.5,CONE
5,10002,"""SORBET"" Lychee",2018-01-01 12:23:09,3.0,1,3.0,SORBET
...,...,...,...,...,...,...,...
29817,19997,"""CONE"" Waffle Cone",2018-07-28 17:40:40,4.0,3,12.0,CONE
29818,19997,"""SORBET"" Blood Orange",2018-07-28 17:40:40,2.5,3,7.5,SORBET
29819,19998,"""SORBET"" Lychee",2018-07-28 18:21:44,3.0,1,3.0,SORBET
29820,19998,"""ICE CREAM"" Rocky Road",2018-07-28 18:21:44,3.5,1,3.5,ICE CREAM


In [ ]:
df.head()

#### Analysis, finally!

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
df.groupby('name')['line_total'].sum().sort_values(ascending=False).head(10).plot(kind='bar')
f.autofmt_xdate()
plt.show()